In [103]:
#6: Use World Bank population data (bigquery-public-data.world_bank_global_population.population_by_country). 
# Query entire data set to your computer using BigQuery API. 
# Transform the data in Python to the format where the first column is year, other columns are populations in countries. 
# Create table for population in Nordics (headers will be ['year','Finland','Sweden','Norway','Denmark','Iceland’]). 
# Upload this table to BigQuery using API.

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
 

In [105]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/tuukka/gcp_keys/atlantean-stock-385411-fce448540d0c.json"
print(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

/home/tuukka/gcp_keys/atlantean-stock-385411-fce448540d0c.json


In [106]:
projectid = "atlantean-stock-385411"

sql = """
SELECT *
FROM `bigquery-public-data.world_bank_global_population.population_by_country`
"""

df_raw = pandas_gbq.read_gbq(sql, project_id=projectid)
print(df_raw.head())

                     country country_code  year_1960  year_1961  year_1962  \
0  Sint Maarten (Dutch part)          SXM       <NA>       <NA>       <NA>   
1             Not classified          INX       <NA>       <NA>       <NA>   
2         West Bank and Gaza          PSE       <NA>       <NA>       <NA>   
3                     Serbia          SRB       <NA>       <NA>       <NA>   
4                   Cambodia          KHM    5722370    5872966    6028431   

   year_1963  year_1964  year_1965  year_1966  year_1967  ...  year_2010  \
0       <NA>       <NA>       <NA>       <NA>       <NA>  ...      34056   
1       <NA>       <NA>       <NA>       <NA>       <NA>  ...       <NA>   
2       <NA>       <NA>       <NA>       <NA>       <NA>  ...    3786161   
3       <NA>       <NA>       <NA>       <NA>       <NA>  ...    7291436   
4    6183584    6331449    6467197    6585035    6685960  ...   14312212   

   year_2011  year_2012  year_2013  year_2014  year_2015  year_2016  \
0  

/home/tuukka/data_engineering/venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2689: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [107]:
#Transform the data in Python to the format where the first column is year, 
# other columns are populations in countries. 
df_raw.columns

Index(['country', 'country_code', 'year_1960', 'year_1961', 'year_1962',
       'year_1963', 'year_1964', 'year_1965', 'year_1966', 'year_1967',
       'year_1968', 'year_1969', 'year_1970', 'year_1971', 'year_1972',
       'year_1973', 'year_1974', 'year_1975', 'year_1976', 'year_1977',
       'year_1978', 'year_1979', 'year_1980', 'year_1981', 'year_1982',
       'year_1983', 'year_1984', 'year_1985', 'year_1986', 'year_1987',
       'year_1988', 'year_1989', 'year_1990', 'year_1991', 'year_1992',
       'year_1993', 'year_1994', 'year_1995', 'year_1996', 'year_1997',
       'year_1998', 'year_1999', 'year_2000', 'year_2001', 'year_2002',
       'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007',
       'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017',
       'year_2018', 'year_2019'],
      dtype='object')

In [108]:
#Transpose dataframe
df_t = df_raw.transpose().reset_index()

In [109]:
#Rename columns as countries
df_t.columns = df_t.iloc[0,:]
#drop country and country code rows
df_t = df_t[2:]
df_t = df_t.reset_index(drop = True)

In [ ]:
#Form separate year column as integer
df_t['year'] = df_t['country'].str[5:].astype(int)


In [120]:
#Create table for population in Nordics (headers will be ['year','Finland','Sweden','Norway','Denmark','Iceland’]). Upload this table to BigQuery using API.
df_nordics = df_t[['year','Finland','Sweden','Norway','Denmark','Iceland']]

In [124]:
pandas_gbq.to_gbq(df_nordics, destination_table = 'data_engineering.nordic_population', project_id = projectid, if_exists='replace')